In [9]:
import pandas as pd

# Load the Excel file
file_path = 'test_data.xlsx'
data_df = pd.read_excel(file_path, sheet_name='data')
labels_df = pd.read_excel(file_path, sheet_name='labels')

# Display the first few rows of each sheet
data_df.head(), labels_df.head(), data_df.shape, labels_df.shape

(  Sample    F1     F2    F3    F4    F5    F6    F7    F8     F9  ...   F77  \
 0     S1  3.65   9.54  3.14  3.48  4.73  3.00  3.06  3.89   3.49  ...  2.96   
 1     S2  3.61   3.78  3.15   NaN  3.34  6.72  3.07  8.31   3.03  ...   NaN   
 2     S3  3.33  10.08  2.92  5.60  5.73  2.92   NaN  4.05  10.56  ...  2.90   
 3     S4  6.71   7.24  3.14  3.47   7.3  2.86  3.14  5.22   3.62  ...  2.91   
 4     S5  5.02   8.74  3.25  8.89  8.06  2.92  3.13   5.6  10.99  ...  2.90   
 
      F78   F79   F80   F81   F83   F85    Z1     Z2    Z3  
 0  11.73  2.89  8.34  3.01  9.12  2.89  3.28  12.09  8.83  
 1   3.21  2.84  3.38  3.62  2.79  3.32  6.96  11.66  7.73  
 2   2.87  2.68  3.30  2.83   2.8  2.73   7.6    NaN  7.68  
 3   2.91  2.82  3.45  3.06  2.66  3.08  3.59  10.67  7.64  
 4   2.68  2.68  3.45  2.88  2.92  2.90  6.92  12.87  7.59  
 
 [5 rows x 86 columns],
   Sample   Label
 0     S1  Group0
 1     S2  Group1
 2     S3  Group1
 3     S4  Group1
 4     S5  Group1,
 (804, 86),
 (804

In [8]:
# Checking for missing values
print("Missing values per column in data:")
print(data_df.isnull().sum())

print("Missing values per column in labels:")
print(labels_df.isnull().sum())


Missing values per column in data:
Sample     0
F1        41
F2        47
F3        30
F4        49
          ..
F83       45
F85       43
Z1        39
Z2        42
Z3        42
Length: 86, dtype: int64
Missing values per column in labels:
Sample    0
Label     0
dtype: int64


Here there is no missing values in the labels. But the nummber of null values in the dataset is quite high. Because almost every column has nul values. So we need to take care of it by imputation. we can not just delete the rows because the number of null values are significant (around 40 rows per column out of 804 rows)

Before imputation lets the column datatypes.

In [15]:
# Checking for data types
print("Data types in data:")
datatypes = data_df.dtypes
print(datatypes.value_counts())

Data types in data:
float64    47
object     39
Name: count, dtype: int64


we can see 47 of the columns have ddata types of float. and 39 have datatypes of object. when i checked the head of the dataset it seemed every  column had float values. 

But as we are getting object types for some columns that meeans there are some non numberic values in those colummns. lets check for non numeric values then.

In [4]:
# Define the columns to check for non-numeric values
columns_to_check = data_df.columns[1:]

# Initialize a dictionary to store non-numeric values
non_numeric_values = {}

# Iterate over the specified columns
for col in columns_to_check:
    # Use pd.to_numeric with errors='coerce' to convert the column to numeric, coercing errors to NaN
    # Then, check for NaN values that indicate non-numeric entries
    non_numeric_mask = pd.to_numeric(data_df[col], errors='coerce').isna()
    
    # Extract non-numeric values
    if non_numeric_mask.any():
        non_numeric_values[col] = data_df.loc[non_numeric_mask, col].unique()

# Print non-numeric values if found
if non_numeric_values:
    for col, values in non_numeric_values.items():
        print(f"Non-numeric values found in column '{col}': {values}")
else:
    print("No non-numeric values found in specified columns.")

Non-numeric values found in column 'F1': [nan]
Non-numeric values found in column 'F2': [nan]
Non-numeric values found in column 'F3': [nan '#value']
Non-numeric values found in column 'F4': [nan]
Non-numeric values found in column 'F5': [nan '#value']
Non-numeric values found in column 'F6': [nan]
Non-numeric values found in column 'F7': [nan '#value']
Non-numeric values found in column 'F8': [nan '#value']
Non-numeric values found in column 'F9': [nan '#value']
Non-numeric values found in column 'F10': [nan '#value']
Non-numeric values found in column 'F11': [nan '#value']
Non-numeric values found in column 'F12': [nan '#value']
Non-numeric values found in column 'F13': [nan '#value']
Non-numeric values found in column 'F14': [nan]
Non-numeric values found in column 'F15': [nan]
Non-numeric values found in column 'F16': [nan]
Non-numeric values found in column 'F17': [nan '#value']
Non-numeric values found in column 'F18': [nan]
Non-numeric values found in column 'F19': [nan '#value'

if we check the above output then we can see the non numeric value that is creting problem is "#value". 

now lets check how value times this appears in each column. if the number is very low. we can just delete the rows.

In [16]:
# Define the columns to check for non-numeric values
columns_to_check = data_df.columns[1:]

# Initialize a dictionary to store counts of "#value"
value_counts = {}

# Iterate over the specified columns
for col in columns_to_check:
    # Count the occurrences of "#value"
    count = (data_df[col] == '#value').sum()
    
    if count > 0:
        value_counts[col] = count

# Print counts if found
if value_counts:
    for col, count in value_counts.items():
        print(f"'#value' found in column '{col}' {count} times.")
else:
    print("No '#value' found in specified columns.")

'#value' found in column 'F3' 1 times.
'#value' found in column 'F5' 1 times.
'#value' found in column 'F7' 1 times.
'#value' found in column 'F8' 1 times.
'#value' found in column 'F9' 1 times.
'#value' found in column 'F10' 1 times.
'#value' found in column 'F11' 1 times.
'#value' found in column 'F12' 1 times.
'#value' found in column 'F13' 1 times.
'#value' found in column 'F17' 1 times.
'#value' found in column 'F19' 1 times.
'#value' found in column 'F21' 1 times.
'#value' found in column 'F24' 1 times.
'#value' found in column 'F25' 1 times.
'#value' found in column 'F29' 1 times.
'#value' found in column 'F32' 1 times.
'#value' found in column 'F35' 1 times.
'#value' found in column 'F36' 1 times.
'#value' found in column 'F38' 1 times.
'#value' found in column 'F40' 1 times.
'#value' found in column 'F48' 1 times.
'#value' found in column 'F51' 1 times.
'#value' found in column 'F52' 1 times.
'#value' found in column 'F53' 1 times.
'#value' found in column 'F56' 1 times.
'#val

we can see "#value" occures only once in each columns that has it. so we can just remove the rows and then see if it cause any problem.

before deleting i am merging the data and labels in one dataframe so that no incosistency happens.

In [5]:
# Define the columns to check for non-numeric values
columns_to_check = data_df.columns[1:]

# Initialize a dictionary to store non-numeric values
non_numeric_values = {}

# Iterate over the specified columns
for col in columns_to_check:
    # Use pd.to_numeric with errors='coerce' to convert the column to numeric, coercing errors to NaN
    # Then, check for NaN values that indicate non-numeric entries
    non_numeric_mask = pd.to_numeric(data_df[col], errors='coerce').isna()
    
    # Extract non-numeric values
    if non_numeric_mask.any():
        # Filter out 'nan' values
        non_numeric_entries = data_df.loc[non_numeric_mask, col]
        non_numeric_entries = non_numeric_entries[non_numeric_entries != 'nan']
        
        if non_numeric_entries.any():
            non_numeric_values[col] = non_numeric_entries.unique()

# Print non-numeric values if found
if non_numeric_values:
    for col, values in non_numeric_values.items():
        print(f"Non-numeric values found in column '{col}': {values}")
else:
    print("No non-numeric values found in specified columns.")

Non-numeric values found in column 'F3': [nan '#value']
Non-numeric values found in column 'F5': [nan '#value']
Non-numeric values found in column 'F7': [nan '#value']
Non-numeric values found in column 'F8': [nan '#value']
Non-numeric values found in column 'F9': [nan '#value']
Non-numeric values found in column 'F10': [nan '#value']
Non-numeric values found in column 'F11': [nan '#value']
Non-numeric values found in column 'F12': [nan '#value']
Non-numeric values found in column 'F13': [nan '#value']
Non-numeric values found in column 'F17': [nan '#value']
Non-numeric values found in column 'F19': [nan '#value']
Non-numeric values found in column 'F21': [nan '#value']
Non-numeric values found in column 'F24': [nan '#value']
Non-numeric values found in column 'F25': [nan '#value']
Non-numeric values found in column 'F29': [nan '#value']
Non-numeric values found in column 'F32': [nan '#value']
Non-numeric values found in column 'F35': [nan '#value']
Non-numeric values found in column '